<a href="https://colab.research.google.com/github/TheOneTrueGuy/Janus/blob/main/Janus_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deepseek-ai/Janus.git


Cloning into 'Janus'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 4), reused 35 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 955.66 KiB | 6.73 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
%cd Janus
!pip install -e .
!pip install flash-attn

/content/Janus
Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for janus (pyproject.toml) ... done
  Created wheel for janus: filename=janus-1.0.0-0.editable-py3-none-any.whl size=13213 sha256=6549905c4ca7d80ffe93b29502638be9402c9b6e9ad02be1fe7c1de49807f601
  Stored in directory: /tmp/pip-ephem-wheel-cache-3bjwhhza/wheels/db/43/fb/dfa5a5a63d3fcf28c66693ac6be0a596a4e95ee0b525246235
Successfully built janus
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9

In [ ]:

import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/106k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

...

Add image tag = <image_placeholder> to the tokenizer


config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

User: <image_placeholder>
Convert the formula into latex code.

Assistant: Sure, here is the LaTeX code for the formula:

\[ A_n = a_0 \left[ 1 + \frac{3}{4} \sum_{k=1}^{n} \left( \frac{4}{9} \right)^k \right] \]


In [1]:
!git clone https://github.com/deepseek-ai/Janus.git


Cloning into 'Janus'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 115 (delta 23), reused 22 (delta 14), pack-reused 74 (from 1)
Receiving objects: 100% (115/115), 7.16 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
%cd Janus
!pip install -e .
!pip install flash-attn

/content/Janus
Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for janus (pyproject.toml) ... done
  Created wheel for janus: filename=janus-1.0.0-0.editable-py3-none-any.whl size=15926 sha256=60131b36f12876536017cac736bb2833ae67c7106eb2f0def9b2cad00a85fd21
  Stored in directory: /tmp/pip-ephem-wheel-cache-mglltsjm/wheels/04/ee/d6/76a460ef4080a263aa86cc3fdbb1c5bb29f559fbd8155d1c83
Successfully built janus
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.3-cp311-cp311-linux_x86_64.whl size=191363917 sha256=b1243e9b86687348a5ab03a073abacdf8e3d5e9e4b7e5326a183f47348c5dfba
  Stored in directory: /root/.cache/pip/wheels/d0/a3/f9/48d2706cb2eac05ec0dc144bf6954

In [3]:

import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.float32).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


Some kwargs in processor config are unused and will not have any effect: add_special_token, sft_format, ignore_id, image_tag, num_image_tokens, mask_prompt. 


Add image tag = <image_placeholder> to the tokenizer


RuntimeError: Input type (c10::BFloat16) and bias type (float) should be the same

In [1]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
# Change the model dtype to torch.float32 to match the bias type
vl_gpt = vl_gpt.to(torch.float32).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:524: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) b

Add image tag = <image_placeholder> to the tokenizer


Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


RuntimeError: Input type (c10::BFloat16) and bias type (float) should be the same

In [2]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
# Change the model dtype to torch.float32 to match the bias type
vl_gpt = vl_gpt.to(auto).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Some kwargs in processor config are unused and will not have any effect: add_special_token, ignore_id, num_image_tokens, mask_prompt, image_tag, sft_format. 


Add image tag = <image_placeholder> to the tokenizer


NameError: name 'auto' is not defined

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
# Change the model dtype to torch.float32 to match the bias type
vl_gpt = vl_gpt.to(torch.float32).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Some kwargs in processor config are unused and will not have any effect: add_special_token, ignore_id, num_image_tokens, mask_prompt, image_tag, sft_format. 


Add image tag = <image_placeholder> to the tokenizer


In [ ]:
#!pip install accelerate

import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# Initialize with empty weights to avoid loading the full model into memory
with init_empty_weights():
    vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
        model_path, trust_remote_code=True
    )

# Load the model using accelerate to distribute it across devices
vl_gpt = load_checkpoint_and_dispatch(
    vl_gpt, model_path, device_map="auto", no_split_module_classes=["CLIPVisionTower"]
)
vl_gpt.eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

In [3]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, device_map="auto", trust_remote_code=True
)
# Change the model dtype to torch.float32 to match the bias type
vl_gpt = vl_gpt.to(torch.float32).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["/content/Janus/images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

Some kwargs in processor config are unused and will not have any effect: num_image_tokens, mask_prompt, image_tag, ignore_id, add_special_token, sft_format. 


Add image tag = <image_placeholder> to the tokenizer


RuntimeError: Input type (c10::BFloat16) and bias type (float) should be the same

In [4]:
from transformers import pipeline

vqa = pipeline(model="deepseek-ai/Janus-1.3B")
output = vqa(
    image="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png",
    question="What is the invoice number?",
)
output[0]["score"] = round(output[0]["score"], 3)
output

KeyError: "Unknown task any-to-any, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [5]:
from transformers import pipeline

# Specify the task explicitly when creating the pipeline
vqa = pipeline(model="deepseek-ai/Janus-1.3B", task="visual-question-answering")

output = vqa(
    image="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png",
    question="What is the invoice number?",
)
output[0]["score"] = round(output[0]["score"], 3)
output

ValueError: Could not load model deepseek-ai/Janus-1.3B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForVisualQuestionAnswering'>,). See the original errors:

while loading with AutoModelForVisualQuestionAnswering, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 567, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'janus.models.modeling_vlm.MultiModalityConfig'> for this kind of AutoModel: AutoModelForVisualQuestionAnswering.
Model type should be one of BlipConfig, Blip2Config, ViltConfig.




In [8]:
from transformers import pipeline

vqa = pipeline("image-to-text", model="deepseek-ai/Janus-1.3B")

output = vqa(
    image="https://huggingface.co/spaces/impira/docquery/resolve/2359223c1837a7587402bda0f2643382a6eefeab/invoice.png",
    question="What is the invoice number?",
)
output[0]["score"] = round(output[0]["score"], 3)
output

ValueError: Could not load model deepseek-ai/Janus-1.3B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForVision2Seq'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForVision2Seq'>). See the original errors:

while loading with AutoModelForVision2Seq, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 567, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'janus.models.modeling_vlm.MultiModalityConfig'> for this kind of AutoModel: AutoModelForVision2Seq.
Model type should be one of BlipConfig, Blip2Config, ChameleonConfig, GitConfig, Idefics2Config, Idefi

In [9]:
from transformers import pipeline

# Specify the task to be 'image-to-text'
vision_classifier = pipeline(model="deepseek-ai/Janus-1.3B", task="image-to-text")

preds = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
# Print the generated text directly
print(preds[0]['generated_text'])

ValueError: Could not load model deepseek-ai/Janus-1.3B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForVision2Seq'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForVision2Seq'>). See the original errors:

while loading with AutoModelForVision2Seq, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 567, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'janus.models.modeling_vlm.MultiModalityConfig'> for this kind of AutoModel: AutoModelForVision2Seq.
Model type should be one of BlipConfig, Blip2Config, ChameleonConfig, GitConfig, Idefics2Config, Idefi

In [7]:
from transformers import pipeline

vision_classifier = pipeline(model="deepseek-ai/Janus-1.3B")
preds = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

KeyError: "Unknown task any-to-any, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [10]:
from transformers import pipeline

# This model is a `zero-shot-classification` model.
# It will classify text, except you are free to choose any label you might imagine
classifier = pipeline(model="deepseek-ai/Janus-1.3B")
classifier(
    "I have a problem with my iphone that needs to be resolved asap!!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
)

KeyError: "Unknown task any-to-any, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [11]:
from transformers import pipeline, AutoProcessor, VisionEncoderDecoderModel

model_name = "deepseek-ai/Janus-1.3B"
processor = AutoProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

vision_classifier = pipeline("image-to-text", model=model, processor=processor)

preds = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
print(preds[0]['generated_text'])

You are using a model of type multi_modality to instantiate a model of type vision-encoder-decoder. This is not supported for all configurations of models and can yield errors.


ValueError: A configuraton of type multi_modality cannot be instantiated because not both `encoder` and `decoder` sub-configurations are passed, but only {'aligner_config': {'cls': 'MlpProjector', 'model_type': 'aligner', 'params': {'depth': 2, 'input_dim': 1024, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_aligner_config': {'cls': 'MlpProjector', 'model_type': 'gen_aligner', 'params': {'depth': 2, 'input_dim': 8, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_head_config': {'cls': 'vision_head', 'model_type': 'gen_head', 'params': {'image_token_embed': 2048, 'image_token_size': 16384, 'n_embed': 2048}}, 'gen_vision_config': {'cls': 'VQ-16', 'model_type': 'gen_vision', 'params': {'image_token_size': 16384, 'n_embed': 8}}, 'language_config': {'hidden_size': 2048, 'intermediate_size': 5632, 'max_position_embeddings': 16384, 'model_type': 'llama', 'num_attention_heads': 16, 'num_hidden_layers': 24, 'num_key_value_heads': 16, 'torch_dtype': 'bfloat16', 'vocab_size': 102400,

In [ ]:
from transformers import pipeline, AutoProcessor
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from janus.models import MultiModalityCausalLM

model_name = "deepseek-ai/Janus-1.3B"
processor = AutoProcessor.from_pretrained(model_name)

# Initialize with empty weights to avoid loading the full model into memory
with init_empty_weights():
    model = MultiModalityCausalLM.from_pretrained(
        model_name, trust_remote_code=True
    )

# Load the model using accelerate to distribute it across devices
model = load_checkpoint_and_dispatch(
    model, model_name, device_map="auto", no_split_module_classes=["CLIPVisionTower"]
)
model.eval()

vision_classifier = pipeline("image-to-text", model=model, processor=processor)

preds = vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)
print(preds[0]['generated_text'])

In [14]:
!pip install pytesseract

ayhgشغال

In [20]:
from PIL import Image
import pytesseract

# فتح الصورة
image = Image.open("/content/Q6rhyjehOXufeUM97MXjX.webp")

# استخراج النص من الصورة باستخدام OCR
extracted_text = pytesseract.image_to_string(image)

print("Extracted Text: ", extracted_text)


Extracted Text:  lows That's
Fit to Print.”

 

 

 

 

 

 

 

 

 

 

 

iy Tak Tbe San a aan Farm

 

 

 

TITANIC SINKS FOUR HOURS AFTER HITTING . ICEBERG;
866 RESCUED BY CARPATHIA, PROBABLY 1250 PERISH;
ISMAY SAFE, MRS. AS AS TOR IR MA YBE, NOTED NAMES MISSING

 

Col, Astor and Bride,
Isidor Straus and Wife,
‘and Wai, Butt Aboard.

‘RULE OF SEA” FOL.)

 

  

[FRANKLIN HOPEFUL ALL OY,

 

 

 

 

 

 

 

 

 
 
 
 
  
 
  
    

G] |biczest Liner Punges

to the Bottom
at 220 A.M.

RESGUERS THERE TOO LATE

xcept to Pek Up he Fo Hone

 

 

 



In [16]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,190 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124788 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [17]:
!apt-get install tesseract-ocr-eng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [18]:
!export PATH="$PATH:/usr/bin/tesseract"

In [19]:
!export PATH="$PATH:/usr/local/Cellar/tesseract/<version>/bin"  # Replace <version> with your Tesseract version

In [ ]:
# مثال للتشغيل باستخدام عنوان تفصيلي
run_example("<DETAILED_CAPTION>")

# مثال للتشغيل باستخدام عنوان أكثر تفصيلاً
run_example("<MORE_DETAILED_CAPTION>")


In [ ]:
!pip install pytesseract

In [ ]:
from PIL import Image
import pytesseract

# فتح الصورة
image = Image.open("/content/Q6rhyjehOXufeUM97MXjX.webp")

# استخراج النص من الصورة باستخدام OCR
extracted_text = pytesseract.image_to_string(image)

print("Extracted Text: ", extracted_text)


Extracted Text:  lows That's
Fit to Print.”

 

 

 

 

 

 

 

 

 

 

 

iy Tak Tbe San a aan Farm

 

 

 

TITANIC SINKS FOUR HOURS AFTER HITTING . ICEBERG;
866 RESCUED BY CARPATHIA, PROBABLY 1250 PERISH;
ISMAY SAFE, MRS. AS AS TOR IR MA YBE, NOTED NAMES MISSING

 

Col, Astor and Bride,
Isidor Straus and Wife,
‘and Wai, Butt Aboard.

‘RULE OF SEA” FOL.)

 

  

[FRANKLIN HOPEFUL ALL OY,

 

 

 

 

 

 

 

 

 
 
 
 
  
 
  
    

G] |biczest Liner Punges

to the Bottom
at 220 A.M.

RESGUERS THERE TOO LATE

xcept to Pek Up he Fo Hone

 

 

 



In [ ]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,190 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124788 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!apt-get install tesseract-ocr-eng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!export PATH="$PATH:/usr/bin/tesseract"

In [21]:
!export PATH="$PATH:/usr/local/Cellar/tesseract/<version>/bin"  # Replace <version> with your Tesseract version

In [ ]:
# مثال للتشغيل باستخدام عنوان تفصيلي
run_example("<DETAILED_CAPTION>")

# مثال للتشغيل باستخدام عنوان أكثر تفصيلاً
run_example("<MORE_DETAILED_CAPTION>")


In [22]:
run_example("/content/Q6rhyjehOXufeUM97MXjX.webp")

NameError: name 'run_example' is not defined

In [ ]:
from PIL import Image
from transformers import pipeline

vqa_pipeline = pipeline("visual-question-answering")

image =  Image.open("/content/Q6rhyjehOXufeUM97MXjX.webp")
question = "Is there an elephant?"

vqa_pipeline(image, question, top_k=1)

In [ ]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor

model = Pix2StructForConditionalGeneration.from_pretrained(PATH_TO_SAVE)
processor = Pix2StructProcessor.from_pretrained(PATH_TO_SAVE)

model.push_to_hub("USERNAME/MODEL_NAME")
processor.push_to_hub("USERNAME/MODEL_NAME")


In [23]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

processor = Pix2StructProcessor.from_pretrained('deepseek-ai/Janus-1.3B')
model = Pix2StructForConditionalGeneration.from_pretrained('deepseek-ai/Janus-1.3B')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'T5TokenizerFast'.
Some kwargs in processor config are unused and will not have any effect: num_image_tokens, mask_prompt, image_tag, ignore_id, add_special_token, sft_format. 


TypeError: Pix2StructProcessor.__init__() got an unexpected keyword argument 'add_special_token'

In [1]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

# Load the processor without specifying the pretrained model
processor = Pix2StructProcessor()
model = Pix2StructForConditionalGeneration.from_pretrained('deepseek-ai/Janus-1.3B')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))

TypeError: Pix2StructProcessor.__init__() missing 2 required positional arguments: 'image_processor' and 'tokenizer'

In [2]:
from transformers import AutoProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

# Load the appropriate processor for Janus-1.3B
processor = AutoProcessor.from_pretrained('deepseek-ai/Janus-1.3B')
model = VisionEncoderDecoderModel.from_pretrained('deepseek-ai/Janus-1.3B')

url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

# Adjust the input format to match the processor's expectations
inputs = processor(images=image, text=["Generate underlying data table of the figure below:"], return_tensors="pt")

predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.batch_decode(predictions, skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - 

ValueError: A configuraton of type multi_modality cannot be instantiated because not both `encoder` and `decoder` sub-configurations are passed, but only {'aligner_config': {'cls': 'MlpProjector', 'model_type': 'aligner', 'params': {'depth': 2, 'input_dim': 1024, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_aligner_config': {'cls': 'MlpProjector', 'model_type': 'gen_aligner', 'params': {'depth': 2, 'input_dim': 8, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_head_config': {'cls': 'vision_head', 'model_type': 'gen_head', 'params': {'image_token_embed': 2048, 'image_token_size': 16384, 'n_embed': 2048}}, 'gen_vision_config': {'cls': 'VQ-16', 'model_type': 'gen_vision', 'params': {'image_token_size': 16384, 'n_embed': 8}}, 'language_config': {'hidden_size': 2048, 'intermediate_size': 5632, 'max_position_embeddings': 16384, 'model_type': 'llama', 'num_attention_heads': 16, 'num_hidden_layers': 24, 'num_key_value_heads': 16, 'torch_dtype': 'bfloat16', 'vocab_size': 102400,

In [3]:
#!pip install accelerate
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
import requests
from PIL import Image
import torch

# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"

# Initialize processor and tokenizer
processor = AutoProcessor.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Initialize with empty weights to avoid loading the full model into memory
with init_empty_weights():
    vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
        model_path, trust_remote_code=True
    )

# Load the model using accelerate to distribute it across devices
vl_gpt = load_checkpoint_and_dispatch(
    vl_gpt, model_path, device_map="auto", no_split_module_classes=["CLIPVisionTower"]
)
vl_gpt.eval()

# Download and open the image
url = "https://raw.githubusercontent.com/vis-nlp/ChartQA/main/ChartQA%20Dataset/val/png/5090.png"
image = Image.open(requests.get(url, stream=True).raw)

# Prepare inputs
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nGenerate underlying data table of the figure below:",
        "images": [image], # Pass the image directly
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
#pil_images = load_pil_images(conversation)
prepare_inputs = processor(
    conversations=conversation, images=image, force_batchify=True
).to(vl_gpt.device)


# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(answer)

Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:524: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in LlamaForCausalLM is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2400: UserWarning: for vision_model.vision_tower.pos_embed: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean 

ValueError: `checkpoint` should be the path to a file containing a whole state dict, or the index of a sharded checkpoint, or a folder containing a sharded checkpoint or the whole state dict, but got deepseek-ai/Janus-1.3B.

In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor
import torch
from PIL import Image

# تحميل النموذج
model_name = "facebook/nougat-base"
model = VisionEncoderDecoderModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# نقل النموذج إلى GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def extract_formula(image_path):
    # تحميل وتجهيز الصورة
    image = Image.open(image_path)
    pixel_values = feature_extractor(image, return_tensors="pt").pixel_values.to(device)

    # توليد النص
    outputs = model.generate(
        pixel_values,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    # فك ترميز النتيجة
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# استخدام النموذج
image_path = "images/equation.png"
latex_formula = extract_formula(image_path)
print("LaTeX formula:", latex_formula)

In [1]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor
import torch
from PIL import Image

# تحميل النموذج
model_name = "deepseek-ai/Janus-1.3B"
model = VisionEncoderDecoderModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# نقل النموذج إلى GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def extract_formula(image_path):
    # تحميل وتجهيز الصورة
    image = Image.open(image_path)
    pixel_values = feature_extractor(image, return_tensors="pt").pixel_values.to(device)

    # توليد النص
    outputs = model.generate(
        pixel_values,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    # فك ترميز النتيجة
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# استخدام النموذج
image_path = "images/equation.png"
latex_formula = extract_formula(image_path)
print("LaTeX formula:", latex_formula)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type multi_modality to instantiate a model of type vision-encoder-decoder. This is not supported for all configurations of models and can yield errors.


ValueError: A configuraton of type multi_modality cannot be instantiated because not both `encoder` and `decoder` sub-configurations are passed, but only {'aligner_config': {'cls': 'MlpProjector', 'model_type': 'aligner', 'params': {'depth': 2, 'input_dim': 1024, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_aligner_config': {'cls': 'MlpProjector', 'model_type': 'gen_aligner', 'params': {'depth': 2, 'input_dim': 8, 'n_embed': 2048, 'projector_type': 'mlp_gelu'}}, 'gen_head_config': {'cls': 'vision_head', 'model_type': 'gen_head', 'params': {'image_token_embed': 2048, 'image_token_size': 16384, 'n_embed': 2048}}, 'gen_vision_config': {'cls': 'VQ-16', 'model_type': 'gen_vision', 'params': {'image_token_size': 16384, 'n_embed': 8}}, 'language_config': {'hidden_size': 2048, 'intermediate_size': 5632, 'max_position_embeddings': 16384, 'model_type': 'llama', 'num_attention_heads': 16, 'num_hidden_layers': 24, 'num_key_value_heads': 16, 'torch_dtype': 'bfloat16', 'vocab_size': 102400,

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install optimum
!pip install transformers
!pip install torch
!pip install Pillow
!pip install accelerate

In [1]:
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 req

In [1]:
import torch
from transformers import AutoProcessor
from optimum.bettertransformer import BetterTransformer
from PIL import Image

# تحميل النموذج والمعالج
model_name = "deepseek-ai/Janus-1.3B"

# تهيئة المعالج
processor = AutoProcessor.from_pretrained(model_name)

# تحميل النموذج مع إعدادات مخصصة
model = BetterTransformer.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float32,
    trust_remote_code=True,
    use_cache=True
)

def process_image(image_path, prompt):
    # تحميل الصورة
    image = Image.open(image_path)

    # تجهيز المدخلات
    inputs = processor(
        text=prompt,
        images=image,
        return_tensors="pt"
    ).to(model.device)

    # توليد النص
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=1.0,
        top_p=0.9
    )

    # فك ترميز النتيجة
    response = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return response



# استخدام النموذج
image_path = "images/equation.png"
prompt = "Convert the formula into latex code."

response = process_image(image_path, prompt)
print("Response:", response)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: type object 'BetterTransformer' has no attribute 'from_pretrained'

In [2]:
!pip uninstall -y torch
!pip install torch==2.0.1

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.2 MB/s eta 0:00:

In [3]:
!pip uninstall -y transformers
!pip install transformers==4.37.2

Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Successfully uninstalled transformers-4.47.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
janus 1.0.0 requires transformers>=4.38.2, but you have transformers 4.37.2 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [3]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
from optimum.bettertransformer import BetterTransformer
from PIL import Image

# تحميل النموذج والمعالج
model_name = "deepseek-ai/Janus-1.3B"

# تهيئة المعالج
processor = AutoProcessor.from_pretrained(model_name)

# تحميل النموذج الأصلي أولا
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# ثم تحويله باستخدام BetterTransformer
model = BetterTransformer.transform(model, device_map="auto", torch_dtype=torch.float32)
# تم إزالة use_cache لأنه غير مدعوم في transform

# ... (باقي الكود كما هو) ...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: The checkpoint you are trying to load has model type `multi_modality` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [1]:
from PIL import Image
from transformers import AutoTokenizer, AutoModelForVisualQuestionAnswering, pipeline

# تحميل النموذج والتوكنيزر
model_name = "deepseek-ai/Janus-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForVisualQuestionAnswering.from_pretrained(model_name)

# إنشاء الـ pipeline باستخدام النموذج المحمل
vqa_pipeline = pipeline("visual-question-answering", model=model, tokenizer=tokenizer)

# فتح الصورة
image = Image.open("elephant.jpeg")

# السؤال
question = "Is there an elephant?"

# إجراء الإجابة على السؤال
answer = vqa_pipeline(image, question, top_k=1)

# طباعة الإجابة
print(answer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: The checkpoint you are trying to load has model type `multi_modality` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [2]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from PIL import Image

# تثبيت النسخة المناسبة من transformers
!pip install -q transformers==4.36.0
!pip install -q accelerate
!pip install -q safetensors

# تهيئة النموذج بطريقة مختلفة
model_name = "deepseek-ai/Janus-1.3B"

# تحميل التكوين والتخصيص
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
config._attn_implementation = "eager"  # تعطيل FlashAttention
config.torch_dtype = torch.float32

# تحميل النموذج مع التكوين المخصص
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float32,
)

# تحميل التوكنايزر
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_image_and_text(image_path, prompt):
    # تحميل الصورة
    image = Image.open(image_path)

    # تجهيز المدخلات
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    ).to(model.device)

    # إضافة معالجة الصورة
    pixel_values = preprocess_image(image).to(model.device)

    # توليد النص
    outputs = model.generate(
        **inputs,
        images=pixel_values,
        max_new_tokens=512,
        do_sample=False,
        temperature=1.0,
        top_p=0.9,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def preprocess_image(image):
    # تحويل الصورة إلى تنسور
    # (هذه الدالة تحتاج للتعديل حسب متطلبات النموذج)
    return torch.tensor(image)

# استخدام النموذج
image_path = "images/equation.png"
prompt = "Convert the formula into latex code."

try:
    response = process_image_and_text(image_path, prompt)
    print("Response:", response)
except Exception as e:
    print("Error:", str(e))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
janus 1.0.0 requires transformers>=4.38.2, but you have transformers 4.36.0 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.


ValueError: The checkpoint you are trying to load has model type `multi_modality` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [ ]:
!pip uninstall -y transformers

# تثبيت النسخة المناسبة من transformers
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# تثبيت المكتبات الأخرى المطلوبة
!pip install accelerate
!pip install safetensors

In [1]:
# أولاً، إزالة النسخة الحالية من transformers




import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

# تحميل النموذج
model_name = "deepseek-ai/Janus-1.3B"

# تهيئة النموذج مع إعدادات مخصصة
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float32,
    device_map="auto",
    use_safetensors=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# بعد تحميل النموذج، قم بإعادة تشغيل runtime في Colab
# Runtime -> Restart runtime

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `multi_modality` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [2]:
!git clone https://github.com/deepseek-ai/Janus.git # Clone the repository
%cd Janus # Navigate into the cloned repository directory
!pip install -e . # Install the package in editable mode
!pip install flash-attn

Cloning into 'Janus'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 115 (delta 23), reused 22 (delta 14), pack-reused 74 (from 1)
Receiving objects: 100% (115/115), 7.16 MiB | 28.09 MiB/s, done.
Resolving deltas: 100% (48/48), done.
[Errno 2] No such file or directory: 'Janus # Navigate into the cloned repository directory'
/content
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating 

In [5]:
%cd Janus # Navigate into the cloned repository directory
!pip install .

[Errno 2] No such file or directory: 'Janus # Navigate into the cloned repository directory'
/content
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
!pip install torch_xla cloud-tpu-client==0.10 torch==2.0.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
!pip install transformers==4.35.0
!pip install accelerate

In [3]:


import torch
import torch_xla
import torch_xla.core.xla_model as xm
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# تحديد جهاز TPU
device = xm.xla_device()

# تحميل النموذج
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# تهيئة النموذج للعمل على TPU
vl_gpt = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float32  # استخدام float32 بدلاً من bfloat16
)
vl_gpt = vl_gpt.to(device)
vl_gpt.eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# تحميل وإعداد الصور
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(device)

# الحصول على embeddings للصور
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# توليد الاستجابة
with torch.no_grad():
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=False,
        use_cache=True
    )
    # انتظار إكمال العمليات على TPU
    xm.mark_step()

# فك ترميز النتيجة
answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)

ModuleNotFoundError: No module named 'janus'